In [1]:
import os
import sys

# Get the absolute path of the scripts directory
root_path_scripts = os.path.abspath(os.path.join(os.getcwd(), '../scripts/'))

# Add the scripts directory to sys.path
sys.path.append(root_path_scripts)

# Import the function from load-docs.py
from load_docs import load_document 

In [3]:
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
# from langchain.embeddings import SentenceTransformerEmbeddings
# from chromadb.utils import embedding_functions
from langchain.embeddings import HuggingFaceEmbeddings

import openai
from openai import OpenAI

# load env variable
load_dotenv()

openai_client = OpenAI()
openai_api_key = os.getenv('OPENAI_API_KEY')

In [4]:
# Load the document(s)
file_path = 'Robinson Advisory.docx'  # Replace with your actual file path
documents = load_document("../data/Robinson Advisory.docx")

In [7]:

# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)

# Initialize Embedding model
# embeddings = SentenceTransformerEmbeddingFunction()
# embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(documents=split_docs, embedding=embeddings)


In [8]:

# 5. Define the function to augment the query
def augment_query_generated(query, model="gpt-4o-mini"):
    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful expert contract advisor assistant. Provide an example answer to the given question "
                "that might be found in a document like a legal contract."
            )
        },
        {"role": "user", "content": query}
    ]
    response = openai_client.chat.completions.create(
            model=model,
            messages=messages,
        )
    content = response.choices[0].message.content
    return content

In [10]:
# 6. Generate the hypothetical answer and augment the query
original_query = "Is there a non-compete obligation to the Advisor?"

hypothetical_answer = augment_query_generated(original_query)

joint_query = f"{original_query} {hypothetical_answer}"
print(f"Joint Query:\n{joint_query}\n")

# 7. Perform similarity search with the joint query
retrieved_documents = vectorstore.similarity_search(joint_query, k=5)

print("Retrieved Documents:")
for idx, doc in enumerate(retrieved_documents, 1):
    print(f"Document {idx}:\n{doc.page_content}\n")

Joint Query:
Is there a non-compete obligation to the Advisor? **Non-Compete Obligation of the Advisor**

The Advisor agrees that, during the term of this Agreement and for a period of twelve (12) months after the termination of this Agreement, the Advisor shall not, directly or indirectly, engage in any business activities that compete with the Company’s business operations. This includes, but is not limited to, providing advisory services to any entity that is in direct competition with the Company, operating in the same industry, or engaging in any activities that may be deemed detrimental to the interests of the Company.

The Advisor acknowledges that this non-compete obligation is reasonable in scope and duration and is necessary to protect the legitimate business interests of the Company, including but not limited to its confidential information, trade secrets, and established client relationships. Any breach of this non-compete clause shall entitle the Company to seek equitable 

In [12]:
# 8. Define the RAG function
def rag(query, retrieved_documents, model="gpt-4o-mini"):
    information = "\n\n".join([doc.page_content for doc in retrieved_documents])

    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful expert contract advisor assistant. Your users are asking questions about information "
                "contained in a legal contract. Answer the user's question using only the provided information."
            )
        },
        {
            "role": "user",
            "content": f"Question: {query}\n\nInformation:\n{information}\n\nAnswer:"
        }
    ]

    
    response = openai_client.chat.completions.create(
            model=model,
            messages=messages,
        )
    content = response.choices[0].message.content
    return content
# 9. Generate the final answer
Answer = rag(query=original_query, retrieved_documents=retrieved_documents)

print("Answer:")
print(Answer)


Answer:
Yes, there is a non-compete obligation for the Advisor. According to the contract, during the term of engagement with the Company and for a period of 12 months thereafter, the Advisor shall not be involved in any business that competes with the Company’s Business, nor shall they solicit and/or hire any employee and/or service provider of the Company, without the prior written consent of the Company.
